In [1]:
from hashlib import sha256
import datetime
import time
import json

In [2]:
#Eg:
tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}
tx_2 = {
   "addr_from": "5J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "2BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 150.00
}

In [3]:
class Block:      
    def __init__(self, index, transactions, timestamp, previous_hash, nonce=0):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = nonce
        
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

In [13]:
class Blockchain:     
   def __init__(self):
       self.unconfirmed_transactions = []
       self.chain = []
       self.difficulty = 4
       self.create_genesis_block() 
 
     
   def create_genesis_block(self):
        genesis_block = Block(0, ["in the name of allah the most gracious the most merciful ,  First, do no harm"], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
   @property
   def last_block(self):
      return self.chain[-1] 
   
   
   def proof_of_work(self, block):
          block.nonce = 1
          computed_hash = block.compute_hash()
          while not computed_hash.startswith('0' * self.difficulty):
             block.nonce += 1
             computed_hash = block.compute_hash()
          return computed_hash
   
   def is_valid_proof(self, block, block_hash):
        return (block_hash.startswith('0' * self.difficulty) and block_hash == block.compute_hash())
   
   def add_block(self, block, proof):
        previous_hash = self.last_block.hash
        if previous_hash != block.previous_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        block.hash = proof
        self.chain.append(block)
        return True
   def add_new_transaction(self, transaction):
            self.unconfirmed_transactions.append(transaction)    
   
   def mine(self):
        if not self.unconfirmed_transactions:
            return False
 
        last_block = self.last_block
 
        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index
           

In [14]:
Titans = Blockchain()

while len(Titans.chain) <10:
    Titans.add_new_transaction(tx_1)
    Titans.mine()
    print(Titans.chain[-1].hash)
    print(str(Titans.chain[-1].nonce))



00000f0d218dc41c9b31efea8e83a6fe61d86211f3a991885a70783236c42384
16989
00001526b268a6795368abd13c39b094be61a08739d95a61e3b0d0b14fdc96ac
64746
00005f7c98180caf05f917a3bab41800428209c66344f8deb163008d183571a8
119760
00005944ecbff2d7f2eb151339852a5f751d0c0de526693f30f1451595c37ce3
53616
00007b7dc60832c3b49491151462130416ff9cac75318c385781b6da04d4ac55
21859
0000e30887ea6a41e77ca66697ce4bc569ab44825397001d7dfcc123bf5793a7
210919
0000a660494fb79563e6a53b7e10a66a7ca424d394bcb2c9163c251dabb8ca2d
37895
0000b5161679095671fe618d23f2414ff6cefd9ce53a92416c2287d139cbc4b3
107546
0000466d35e43026225cf9b2e58787288369dba965531c996cb8776f9086bcd3
36336


In [ ]:
#Titans.add_new_transaction(tx_1)
#Titans.add_new_transaction(tx_2)